## License 

Copyright 2021 Patrick Hall (jphall@gwu.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

*DISCLAIMER*: This notebook is not legal or compliance advice.

# Model Evaluation Notebook

#### Imports and inits

In [1]:
import os              # for directory and file manipulation
import numpy as np     # for basic array manipulation
import pandas as pd    # for dataframe manipulation
import datetime        # for timestamp

# for model eval
from sklearn.metrics import accuracy_score, f1_score, log_loss, mean_squared_error, roc_auc_score

# global constants 
ROUND = 3              # generally, insane precision is not needed 
SEED = 12345           # seed for better reproducibility

# set global random seed for better reproducibility
np.random.seed(SEED)

#### Set basic metadata

In [2]:
y_name = 'high_priced'
scores_dir = 'data/scores'

#### Read in score files 

In [3]:
# init score frame with known test y values
scores_frame = pd.read_csv(scores_dir + os.sep +'key.csv', index_col='Unnamed: 0')

# create random folds in reproducible way
np.random.seed(SEED)
scores_frame['fold'] = np.random.choice(5, scores_frame.shape[0])

# read in each score file in the directory as a new column 
for file in os.listdir(scores_dir):
    if file != 'key.csv' and file.endswith('.csv'):
        scores_frame[file[:-4]] = pd.read_csv(scores_dir + os.sep + file)['phat']

# sanity check 
scores_frame

,high_priced,fold,group3_randomforest,group4_mxgb,group2_mxgb,group5_ebm,group4_ebm_3,group6_mxgb,ph_mxgb,group6_ebm_2,...,group1_rf,group5_ebm_2,group2_ebm_2,group8_monogbm,group8_ebm_3,ph_ebm,group3_ebm_2,group8_ebm_2,group3_mxgb_2,group4_ebm_2
0,0.0,2,0.063028,0.058037,0.065526,0.079366,0.005820,0.062775,0.059522,0.078887,...,0.100704,0.078818,0.066693,0.169457,0.079292,0.082841,0.171009,0.074440,0.254148,0.063845
1,0.0,1,0.030348,0.032129,0.032689,0.027144,0.254738,0.035047,0.036210,0.027960,...,0.098442,0.029009,0.032970,0.016476,0.028798,0.027079,0.323614,0.026770,0.249009,0.028332
2,1.0,4,0.174309,0.161683,0.167186,0.182317,0.075005,0.161925,0.180734,0.169545,...,0.119991,0.191593,0.177787,0.178777,0.178244,0.190718,0.023242,0.190482,0.033878,0.174635
3,0.0,1,0.019114,0.023556,0.033490,0.029269,0.559781,0.026129,0.027677,0.032026,...,0.065078,0.019538,0.024862,0.009344,0.028750,0.031069,0.015636,0.048819,0.021316,0.022828
4,1.0,2,0.207948,0.180101,0.173838,0.202628,0.066230,0.176872,0.177813,0.193220,...,0.117926,0.203666,0.198582,0.179083,0.203476,0.210361,0.004191,0.211336,0.008621,0.212485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19826,0.0,3,0.245902,0.253574,0.266065,0.225395,0.020803,0.250139,0.274767,0.220331,...,0.124180,0.229297,0.234666,0.247609,0.230922,0.231624,0.035635,0.242678,0.037913,0.222671
19827,0.0,1,0.177097,0.173608,0.184479,0.252359,0.011832,0.181217,0.182039,0.235473,...,0.105582,0.249218,0.259163,0.234628,0.241858,0.254823,0.016384,0.244461,0.023938,0.229508
19828,1.0,3,0.174827,0.229630,0.217184,0.226476,0.328958,0.221794,0.212740,0.198205,...,0.127146,0.224792,0.217782,0.192819,0.211413,0.220400,0.043647,0.213149,0.051774,0.232992
19829,0.0,1,0.035276,0.001175,0.001412,0.001222,0.145872,0.001602,0.001323,0.001106,...,0.056343,0.002591,0.006079,0.003607,0.002935,0.000993,0.027301,0.001347,0.030560,0.008614


#### Utility function for max. accuracy

In [4]:
def max_acc(y, phat, res=0.01): 

    """ Utility function for finding max. accuracy at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. accuracy, default 0.01.
        :return: Max. accuracy for model scores.
    
    """
    
    # init frame to store acc at different cutoffs
    acc_frame = pd.DataFrame(columns=['cut', 'acc'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find accuracy at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        acc = accuracy_score(temp_df.iloc[:, 0], temp_df['decision'])
        acc_frame = acc_frame.append({'cut': cut,
                                      'acc': acc},
                                     ignore_index=True)

    # find max accurcay across all cutoffs
    max_acc = acc_frame['acc'].max()
    
    # house keeping
    del acc_frame, temp_df
    
    return max_acc

####  Utility function for max. F1

In [5]:
def max_f1(y, phat, res=0.01): 
    
    """ Utility function for finding max. F1 at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. F1, default 0.01.
        :return: Max. F1 for model scores.
    
    """
    
    # init frame to store f1 at different cutoffs
    f1_frame = pd.DataFrame(columns=['cut', 'f1'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find f1 at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        f1 = f1_score(temp_df.iloc[:, 0], temp_df['decision'])
        f1_frame = f1_frame.append({'cut': cut,
                                    'f1': f1},
                                    ignore_index=True)
        
    # find max f1 across all cutoffs
    max_f1 = f1_frame['f1'].max()
    
     # house keeping
    del f1_frame, temp_df
    
    return max_f1

#### Rank all submitted scores 

In [6]:
eval_frame = pd.DataFrame() # init frame to hold score ranking
metric_list = ['acc', 'auc', 'f1', 'logloss', 'mse'] # metric to use for evaluation

# create eval frame row-by-row
for fold in sorted(scores_frame['fold'].unique()): # loop through folds 
    for metric_name in metric_list: # loop through metrics
        
        # init row dict to hold each rows values
        row_dict = {'fold': fold,
                    'metric': metric_name}
        
        # cache known y values for fold
        fold_y = scores_frame.loc[scores_frame['fold'] == fold, y_name]
        
        for col_name in scores_frame.columns[2:]:
            
            # cache fold scores
            fold_scores = scores_frame.loc[scores_frame['fold'] == fold, col_name]
            
            # calculate evaluation metric for fold
            # with reasonable precision 
            
            if metric_name == 'acc':
                row_dict[col_name] = np.round(max_acc(fold_y, fold_scores), ROUND)
                
            if metric_name == 'auc':
                row_dict[col_name] = np.round(roc_auc_score(fold_y, fold_scores), ROUND)
                
            if metric_name == 'f1':
                row_dict[col_name] = np.round(max_f1(fold_y, fold_scores), ROUND) 
                
            if metric_name == 'logloss':
                row_dict[col_name] = np.round(log_loss(fold_y, fold_scores), ROUND)
                
            if metric_name == 'mse':
                row_dict[col_name] = np.round(mean_squared_error(fold_y, fold_scores), ROUND)
        
        # append row values to eval_frame
        eval_frame = eval_frame.append(row_dict, ignore_index=True)

# init a temporary frame to hold rank information
rank_names = [name + '_rank' for name in eval_frame.columns if name not in ['fold', 'metric']]
rank_frame = pd.DataFrame(columns=rank_names)        

# set columns to necessary order
eval_frame = eval_frame[['fold', 'metric'] + [name for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]]

# determine score ranks row-by-row
for i in range(0, eval_frame.shape[0]):
        
        # get ranks for row based on metric
        metric_name = eval_frame.loc[i, 'metric']
        if metric_name in ['logloss', 'mse']:
            ranks = eval_frame.iloc[i, 2:].rank().values
        else:
            ranks = eval_frame.iloc[i, 2:].rank(ascending=False).values
        
        # create single-row frame and append to rank_frame
        row_frame = pd.DataFrame(ranks.reshape(1, ranks.shape[0]), columns=rank_names)
        rank_frame = rank_frame.append(row_frame, ignore_index=True)
        
        # house keeping
        del row_frame

# merge ranks onto eval_frame
eval_frame = pd.concat([eval_frame, rank_frame], axis=1)

# house keeping
del rank_frame
        
eval_frame

,fold,metric,group1_ebm,group1_glm,group1_mxgb,group1_rf,group2_ebm,group2_ebm_2,group2_ebm_3,group2_glm,...,group8_ebm_rank,group8_ebm_2_rank,group8_ebm_3_rank,group8_ebmoversample_rank,group8_gbm_2_rank,group8_gbmgrid_rank,group8_monogbm_rank,ph_ebm_rank,ph_glm_rank,ph_mxgb_rank
0,0.0,acc,0.901,0.900,0.901,0.900,0.901,0.901,0.900,0.900,...,18.5,18.5,18.5,38.0,18.5,38.0,18.5,18.5,38.0,3.5
1,0.0,auc,0.839,0.775,0.814,0.803,0.815,0.838,0.837,0.775,...,4.5,10.0,10.0,4.5,1.0,10.0,17.0,4.5,39.5,27.5
2,0.0,f1,0.408,0.335,0.377,0.368,0.379,0.403,0.401,0.335,...,2.0,5.0,10.0,13.5,13.5,17.0,10.0,2.0,38.5,22.0
3,0.0,logloss,0.251,0.291,0.263,0.305,0.263,0.252,0.252,0.291,...,4.5,4.5,12.0,45.0,4.5,4.5,12.0,4.5,35.5,27.0
4,0.0,mse,0.077,0.084,0.079,0.086,0.078,0.077,0.077,0.084,...,10.0,10.0,10.0,45.0,10.0,10.0,10.0,10.0,36.0,23.5
5,1.0,acc,0.906,0.906,0.906,0.906,0.906,0.906,0.907,0.906,...,25.0,25.0,25.0,25.0,2.5,25.0,25.0,25.0,25.0,25.0
6,1.0,auc,0.828,0.757,0.791,0.792,0.793,0.827,0.829,0.757,...,6.5,11.0,15.5,2.5,18.0,2.5,11.0,6.5,39.0,28.5
7,1.0,f1,0.369,0.302,0.339,0.336,0.339,0.373,0.378,0.302,...,9.5,13.5,9.5,4.5,15.5,4.5,18.0,9.5,39.0,32.0
8,1.0,logloss,0.246,0.281,0.263,0.294,0.263,0.247,0.246,0.281,...,3.5,10.0,15.5,45.0,10.0,3.5,15.5,3.5,35.5,30.0
9,1.0,mse,0.074,0.080,0.078,0.082,0.078,0.074,0.074,0.080,...,4.0,13.5,13.5,45.0,13.5,13.5,13.5,4.0,35.5,27.0


#### Save `eval_frame` as CSV

In [7]:
eval_frame.to_csv('model_eval_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                  index=False)

#### Display simple ranked score list 

In [8]:
eval_frame[[name for name in eval_frame.columns if name.endswith('rank')]].mean().sort_values()

group7_EBM_2_rank             8.92
group1_ebm_rank               9.04
ph_ebm_rank                   9.04
group8_ebm_rank               9.04
group2_ebm_3_rank             9.24
group8_gbm_2_rank             9.60
group5_ebm_2_rank            10.94
group8_gbmgrid_rank          10.96
group2_ebm_2_rank            11.88
group5_ebm_rank              11.94
group3_ebm_3_rank            12.16
group4_ebm_2_rank            12.16
group6_ebm_2_rank            12.36
group8_ebm_2_rank            12.58
group8_ebm_3_rank            12.72
group8_monogbm_rank          13.56
group5_ensemble_rank         13.56
group6_ebm_rank              13.84
group5_mgbm_rank             17.04
group7_xgb_2_rank            25.20
group8_ebmoversample_rank    25.32
group3_decisiontree_rank     25.58
group7_xgb_rank              26.30
group7_ebm_rank              26.30
group1_mxgb_rank             26.38
group2_ebm_rank              26.46
group2_mxgb_rank             26.46
group6_mxgb_rank             26.60
group3_randomforest_